# 🧠 Gemini AI Health Monitoring Assistant

This notebook integrates health data with Gemini AI to enable GenAI-powered features like RAG, document summarization, and JSON output generation.

In [1]:
# ✅ Step 1: Install Dependencies
!pip install -q google-generativeai chromadb pandas langchain pymupdf

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.4 which is incompatible.


In [ ]:
# ✅ Step 2: Set up Gemini AI
import google.generativeai as genai

# Replace with your actual Gemini API key
genai.configure(api_key="YOUR_GEMINI_API_KEY")

In [ ]:
# ✅ Step 3: Load Sample Health Data
import pandas as pd

data = {
    'timestamp': pd.date_range(start='2025-04-01', periods=7, freq='D'),
    'heart_rate': [75, 78, 82, 88, 90, 76, 80],
    'body_temp': [36.7, 36.8, 37.1, 37.4, 37.2, 36.9, 37.0],
    'hrv': [70, 65, 60, 58, 59, 66, 68]
}
df = pd.DataFrame(data)
df

In [ ]:
# ✅ Step 4: Embed and Store Data using ChromaDB
import chromadb
from langchain.vectorstores import Chroma
from langchain.embeddings import GoogleGenerativeAIEmbeddings

chroma_client = chromadb.Client()
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

texts = [
    f"On {row['timestamp'].date()}, heart rate was {row['heart_rate']} bpm, temperature was {row['body_temp']}°C, HRV was {row['hrv']} ms"
    for _, row in df.iterrows()
]
docs = [{"id": str(i), "text": t} for i, t in enumerate(texts)]

collection = chroma_client.create_collection(name="health_data")
for doc in docs:
    collection.add(documents=[doc["text"]], ids=[doc["id"]])

print("Embedded documents into vector store.")

In [ ]:
# ✅ Step 5: Ask Health-Related Questions (RAG Style)
query = "How was my health trend over the past week?"
retrieved = collection.query(query_texts=[query], n_results=3)
relevant_text = "\n".join([doc for doc in retrieved["documents"][0]])

response = genai.GenerativeModel("gemini-pro").generate_content(
    f"Based on the following health data:\n{relevant_text}\n\nAnswer this question:\n{query}"
)
print(response.text)

In [ ]:
# ✅ Step 6: Summarize a PDF Report (Optional)
import fitz  # PyMuPDF
from google.colab import files

uploaded = files.upload()
file_path = next(iter(uploaded))

doc = fitz.open(file_path)
text = "\n".join([page.get_text() for page in doc])

summary = genai.GenerativeModel("gemini-pro").generate_content(
    f"Summarize this medical report:\n{text}"
)
print(summary.text)

In [ ]:
# ✅ Step 7: Generate JSON Health Summary (Optional)
response_json = genai.GenerativeModel("gemini-pro").generate_content(
    f"""Based on this health data:\n{relevant_text}\n
    Return a JSON object like this:
    {{
        \"summary\": \"brief summary\",
        \"average_heart_rate\": value,
        \"average_temperature\": value,
        \"hrv_trend\": \"increasing/decreasing\",
        \"status\": \"Healthy/Critical\"
    }}
    """
)
print(response_json.text)